# Thinking 1 ：当我们思考数据源的时候，都有哪些维度，如果你想要使用爬虫抓取数据，都有哪些工具？
>答：数据源大概分为四类：1.公开开放的数据集，如《统计年鉴》等。  
2.可以爬取的网页数据，如，网站内容或者APP内容等，注意robots协议。  
3.传感器数据，如无人驾驶汽车雷达采集信息等，一般是物理信息。  
4.日志信息，用于统计用户的操作，可以前端埋点收集或者后端脚本收集及统计，以统计网站的访问情况及瓶颈等，如网站上一个商品的点击率数据。  

>爬虫工具有两类：一类为可视化工具，以八爪鱼和火车头为代表。特点速度快，模板多，但不全面不够个性化。  
另一类为Python爬虫工具，主要有BeautifulSoup和Selenium两个库，Selenium主要用于模拟人的操作。两者均可以自定义爬取内容。

# Thinking 2 ：企业的数据源来自多个维度，请思考什么是企业的数据中台，你是如何理解一方数据，二方数据，三方数据？
> 答：一方数据主要指自有的数据，以淘宝为例，一方数据为：各个商品的价格，描述，商家所在地等。  
二方数据为媒体交互行为数据，如广告监测，网站分析，舆情分析等，以淘宝为例，二方数据为：各个商品的点击度等。    
三方数据为第三方数据供应商提供的数据，如数据平台，运营商等，以淘宝商家为例，三方数据为：各个商家得到的淘宝提供的信息数据，此处淘宝为第三方数据供应商。  

# Action 1 : 不用任何数学库，如何求出sqrt(10)，并且精确到小数点后10位

In [26]:
# 二分法
def sqrt(num, error):
    max_ = num
    min_ = 0 
    mid = (max_ + min_)/2
    while (mid**2 - num >= error) or (num - mid**2 >= error):
        if mid**2 - num > 0:
            max_ = mid
            mid = (max_ + min_)/2
            
            continue
        else:
            min_ = mid
            mid = (max_ + min_)/2
            
            continue
    return mid    
print(sqrt(10,1e-10))

import numpy as np
print(np.sqrt(10))


3.162277660157997
3.1622776601683795


# Action 2: 汽车投诉信息采集：
数据源：http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml  
投诉编号，投诉品牌，投诉车系，投诉车型，问题简述，典型问题，投诉时间，投诉状态  
可以采用Python爬虫，或者第三方可视化工具  

In [223]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'} 
url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'

#定义爬取的url列表
def get_url_list(num):
    url_list = []
    for i in range(num):
        url_list.append('http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-'+str(i)+'.shtml')
    return url_list 

#爬取一页的函数
def bs_one (url):
    html = requests.get(url, headers = headers, timeout = 10)
    #得到网页的源码
    content = html.text

    #先解析网页
    soup =  bs(content, 'html.parser', from_encoding='utf-8')
    table = soup.find('div', class_ ='tslb_b')
    tr_list = table.find_all('tr')

    #得到Feature
    feature = tr_list[0]
    feature_list = feature.find_all('th')
    feature_list = [i.text for i in feature_list]
    feature_list

    #得到投诉列表
    td_list = [i.find_all('td') for i in tr_list]
    len(td_list)
    table = []
    for i in range(1,len(td_list)):
        td = [j.text for j in td_list[i]]
        table.append(td)
    
    #得到最后的列表    
    data_result = pd.DataFrame(table, columns = feature_list)
    return(data_result)

#定义爬虫
def czw_spider(num):
    url_list = get_url_list(num)
    results = pd.DataFrame()
    for urli in  url_list:
        results = pd.concat([results,bs_one(urli)], ignore_index = True)
    return results


#爬取三页信息
#pd.options.display.max_columns=None
pd.options.display.max_rows=None
results = czw_spider(2)
results

C:\Anaconda\lib\site-packages\bs4\__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,投诉编号,投诉品牌,投诉车系,投诉车型,问题简述,典型问题,投诉时间,投诉状态
0,514343,一汽丰田,普拉多,2018款 3.5L 自动 TX-L 后挂备胎,一汽丰田普拉多发动机无法启动 承诺不兑现,"A44,L313,",2020-08-26,信息审核
1,514342,广汽丰田,凯美瑞,2016款 2.0G 自动 十周年纪念领先版,广汽丰田凯美瑞方向盘脱皮存在质量问题,"H168,",2020-08-26,厂家受理
2,514338,东风本田,本田CR-V,2019款 240TURBO CVT 两驱 风尚版 国V,东风本田CR-V主驾驶门内饰板异响,"H72,",2020-08-26,信息审核
3,514337,沃尔沃（进口）,沃尔沃XC90,2020款 改款 T6 智逸豪华版 7座,沃尔沃XC90低频噪音高达97分贝,"H268,",2020-08-26,信息审核
4,514334,一汽-大众,探岳,2019款 330TSI 两驱 豪华型 国Ⅵ,一汽大众探岳行驶时左前悬架出现异响,"G204,",2020-08-26,信息审核
5,514329,长城汽车,哈弗H2,2014款 1.5T 手动 两驱精英版 5座,哈弗H2开启空调后发动机抖动异响严重,"A9,A24,",2020-08-26,信息审核
6,514322,东风标致,标致308,2013款 1.6L 自动 优尚型,东风标致308真皮座椅破损空调拨片损坏,"H101,H168,H169,H121,",2020-08-26,信息审核
7,514320,奇瑞汽车,瑞虎8,2018款 1.5TCI 自动 豪华型 5座,奇瑞瑞虎8变速箱顿挫 刹车前冲,"B19,E196,",2020-08-26,信息审核
8,514319,北京现代,索纳塔九,2015款 1.6T 双离合 GLS智能型,北京现代索纳塔九质保期内真皮座椅破裂,"H168,I297,",2020-08-26,信息审核
9,514316,一汽解放,虎VH,轻卡,一汽解放虎VH发动机水温高维修未解决,"A191,",2020-08-26,信息审核


# 上述方法不显示具体问题内容，进行改进

In [224]:
#定义可显示原dataframe中一页的具体典型问题的函数
def collect_one_problem(data_result_origin, url):
    #收集典型问题数据
    #原因：html解析文件中不包含此信息 ==》》 使用selenium
    #典型问题开头字母有明确含义，需要进行匹配
    import re
    import numpy as np
    problem_list = []
    for i in data_result_origin['典型问题']:
        problem_codename = re.findall(r'[A-Z]+', i)
        problem_list.append(problem_codename)
#     print(problem_list)
#     print(len(problem_list))

    data_result = pd.DataFrame()
    data_result['服务问题数量'] = [i.count('I')+i.count('J')+i.count('K')+ i.count('L')+i.count('M')+i.count('N')+i.count('O')+i.count('P') for i in problem_list]
    data_result['其他问题数量'] = [i.count('Q') for i in problem_list]
    data_result['基本问题数量'] = [(len(i) - i.count('Q') - (i.count('I')+i.count('J')+i.count('K') +i.count('L')+i.count('M')+i.count('N')+i.count('O')+i.count('P'))) for i in problem_list]
    data_result['基本问题'] = None
    data_result['服务问题'] = None
    data_result['其他问题'] = None
    data_result


    #使用 selenium 得到问题列表
    import json
    import requests
    from lxml import etree
    import time
    from selenium import webdriver

    chrome_driver = 'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'  #chromedriver的文件位置
    # 需要将chromedriver放到Chrome\Application目录下
    driver = webdriver.Chrome(executable_path = chrome_driver)

    # 去空格，去换行\n
    def format_str(str):
        return str.replace('\n', '').replace(' ', '')

    # 对页面进行抓取分析
    #使用xpath Hit ctrl+shift+x ,调取信息（注意：去掉li-over）
    driver.get(url)
    time.sleep(1)
    html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
    html = etree.HTML(html)

    #设置需要抓取的字段
    #统计基本问题
    basic_problem = {}
    problem = {}
    for i in range(30):
        if data_result['基本问题数量'][i] == 1:
            a = i+2
            problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='bw']/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='wt']/a") 
            basic_problem[i] = tuple(j.text for j in problem[i])
            data_result['基本问题'][i] = basic_problem[i]

        if data_result['基本问题数量'][i] >= 1:
            basic_problem[i] = []
            for j in range(data_result['基本问题数量'][i]):
                problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='bw']["+str(j+1)+"]/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='wt']["+str(j+1)+"]/a")
                basic_problem[i].append([g.text for g in problem[i]])
            data_result['基本问题'][i] = basic_problem[i]

    basic_problem = {}
    problem = {}
    for i in range(30):        
        if data_result['服务问题数量'][i] == 1:
            a = i+2
            problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='fwwt']/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='fw']/a") 
            basic_problem[i] = [j.text for j in problem[i]]
            data_result['服务问题'][i] = basic_problem[i]

        if data_result['服务问题数量'][i] >= 1:
            basic_problem[i] = []
            for j in range(data_result['服务问题数量'][i]):
                problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='fwwt']["+str(j+1)+"]/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='fw']["+str(j+1)+"]/a")
                basic_problem[i].append([g.text for g in problem[i]])
            data_result['服务问题'][i] = basic_problem[i]

    basic_problem = {}
    problem = {}
    for i in range(30):        
        if data_result['其他问题数量'][i] == 1:
            a = i+2
            problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='qtwt']/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='qt']/a") 
            basic_problem[i] = [j.text for j in problem[i]]
            data_result['其他问题'][i] = basic_problem[i]

        if data_result['其他问题数量'][i] >= 1:
            basic_problem[i] = []
            for j in range(data_result['其他问题数量'][i]):
                problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='qtwt']["+str(j+1)+"]/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='qt']["+str(j+1)+"]/a")
                basic_problem[i].append([g.text for g in problem[i]])
            data_result['其他问题'][i] = basic_problem[i]

    #拼接得到最后的数据框        
    results = data_result_origin.join(data_result.iloc[:, 3:], how = 'inner')
    return results

#定义更全面的车质网爬虫2
def czw_spider2(num):
    url_list = get_url_list(num)
    results = pd.DataFrame()
    for urli in  url_list:
        data_result_origin = bs_one(urli)
        results = pd.concat([results, collect_one_problem(data_result_origin, urli)], ignore_index = True)
    return results

results = czw_spider2(2)
results

C:\Anaconda\lib\site-packages\bs4\__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,投诉编号,投诉品牌,投诉车系,投诉车型,问题简述,典型问题,投诉时间,投诉状态,基本问题,服务问题,其他问题
0,514343,一汽丰田,普拉多,2018款 3.5L 自动 TX-L 后挂备胎,一汽丰田普拉多发动机无法启动 承诺不兑现,"A44,L313,",2020-08-26,信息审核,"[[发动机, 无法启动]]","[[承诺不兑现, 销售承诺不兑现]]",None
1,514342,广汽丰田,凯美瑞,2016款 2.0G 自动 十周年纪念领先版,广汽丰田凯美瑞方向盘脱皮存在质量问题,"H168,",2020-08-26,厂家受理,"[[车身附件及电器, 部件开裂]]",None,None
2,514338,东风本田,本田CR-V,2019款 240TURBO CVT 两驱 风尚版 国V,东风本田CR-V主驾驶门内饰板异响,"H72,",2020-08-26,信息审核,"[[车身附件及电器, 门窗异响]]",None,None
3,514337,沃尔沃（进口）,沃尔沃XC90,2020款 改款 T6 智逸豪华版 7座,沃尔沃XC90低频噪音高达97分贝,"H268,",2020-08-26,信息审核,"[[车身附件及电器, 车身共振]]",None,None
4,514334,一汽-大众,探岳,2019款 330TSI 两驱 豪华型 国Ⅵ,一汽大众探岳行驶时左前悬架出现异响,"G204,",2020-08-26,信息审核,"[[前后桥及悬挂系统, 悬挂故障]]",None,None
5,514329,长城汽车,哈弗H2,2014款 1.5T 手动 两驱精英版 5座,哈弗H2开启空调后发动机抖动异响严重,"A9,A24,",2020-08-26,信息审核,"[[发动机, 异响], [发动机, 抖动]]",None,None
6,514322,东风标致,标致308,2013款 1.6L 自动 优尚型,东风标致308真皮座椅破损空调拨片损坏,"H101,H168,H169,H121,",2020-08-26,信息审核,"[[车身附件及电器, 空调问题], [车身附件及电器, 部件开裂], [车身附件及电器, 座...",None,None
7,514320,奇瑞汽车,瑞虎8,2018款 1.5TCI 自动 豪华型 5座,奇瑞瑞虎8变速箱顿挫 刹车前冲,"B19,E196,",2020-08-26,信息审核,"[[变速器, 顿挫], [制动系统, 刹车泵故障]]",None,None
8,514319,北京现代,索纳塔九,2015款 1.6T 双离合 GLS智能型,北京现代索纳塔九质保期内真皮座椅破裂,"H168,I297,",2020-08-26,信息审核,"[[车身附件及电器, 部件开裂]]","[[服务态度, 不予索赔]]",None
9,514316,一汽解放,虎VH,轻卡,一汽解放虎VH发动机水温高维修未解决,"A191,",2020-08-26,信息审核,"[[发动机, 冷却系统故障]]",None,None


In [225]:
results.to_csv('车质网.csv', encoding = 'UTF-8-sig')

# 数据的拼接与切片
https://www.cnblogs.com/keye/p/10791705.html  
https://blog.csdn.net/Asher117/article/details/83656584